<a href="https://colab.research.google.com/github/vineeta07/titanicSurvivalPrediction_ml/blob/main/titanic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df_train = pd.read_csv('train.csv')
df_test= pd.read_csv('test.csv')
if'Survived' not in df_test.columns:
    df_test['Survived']=0

In [ ]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
def Preprocess(df_train , df_test):
    df= pd.concat([df_train , df_test] , axis=0)
    df= df.drop(['Name' , 'Ticket' ] , axis=1)
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Cabin']=df['Cabin'].fillna('X000')
    df['Embarked'] =df['Embarked'].fillna('S')
    df['Fare']=df['Fare'].fillna(df['Fare'].mean())

    df['cabin_letter'] = df['Cabin'].str.extract(r'([a-zA-z]+)' ,expand=False)
    df['cabin_number']= df['Cabin'].str.extract(r'(\d+)', expand = False)
    df = df.drop('Cabin' , axis=1)

    df = pd.get_dummies(df, columns=['cabin_letter'] , prefix= 'cabin')
    df = pd.get_dummies(df, columns=['Embarked'] , prefix= 'Embarked')
    df = pd.get_dummies(df, columns=['Sex'] ,prefix='Sex')

    df= df.drop('cabin_X' , axis=1)
    #df= df.drop('Embarked_X' ,axis=1)

    df['cabin_number']= df['cabin_number'].fillna(0)
    df['cabin_number'] = pd.to_numeric(df['cabin_number'])

    df['Pclass_bin_fare'] = df['Fare'] // df['Pclass']
    df['Pclass_bin_sex'] = df['Pclass'] - df['Sex_female']

    df_train= df[:len(df_train)]
    df_test= df[len(df_train):]

    df_test = df_test.drop('Survived' , axis=1)

    return df_train ,df_test

In [ ]:
df= pd.concat([df_train, df_test] , axis=0)
df.isna().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,263
SibSp,0
Parch,0
Ticket,0
Fare,1


In [ ]:
train_df , test_df= Preprocess(df_train , df_test)

In [ ]:
X= train_df.drop('Survived' , axis=1)
y= train_df['Survived']
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2)
y_train = np.reshape(y_train , (-1,1))

In [ ]:
X_train.shape , y_train.shape

((712, 22), (712, 1))

In [25]:
model_1 = LogisticRegression()
model_1.fit(X_train, y_train.ravel())

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [26]:
y_pred = model_1.predict(X_test)

In [27]:
accuracy_score(y_test , y_pred)

0.7877094972067039

In [28]:
model_2= XGBClassifier()
model_2.fit(X_train , y_train.ravel())

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [29]:
y_pred = model_2.predict(X_test)
accuracy_score(y_test , y_pred)

0.7988826815642458

In [30]:
train_df.corr()['Survived']

,Survived
PassengerId,-0.005007
Survived,1.000000
Pclass,-0.338481
Age,-0.070323
SibSp,-0.035322
Parch,0.081629
Fare,0.257307
cabin_number,0.229756
cabin_A,0.022287
cabin_B,0.175095


In [31]:
model_3 = RandomForestClassifier()
model_3.fit(X_train , y_train.ravel())

RandomForestClassifier()

In [32]:
y_pred = model_3.predict(X_test)
accuracy_score(y_test , y_pred)

0.8156424581005587

In [33]:
pred = model_3.predict(test_df)

final = pd.DataFrame()
final['PassengerId']= test_df['PassengerId']
final['Survived']= pred

print(final.shape)
final.to_csv('output.csv' , index=False)


(418, 2)
